# Convert2Mat - 2022

In [96]:
from datetime import datetime
from scipy import io
import obspy
import ooipy
from tqdm import tqdm
import numpy as np

### Create function to automatically save files


In [107]:
def convert2mat(start_time, end_time, nodes, node_names, fn, h_type='bb', verbose=True):
    '''
    convert2mat downloads data using ooipy and converts it to a mat file
    
    Parameters
    ----------
    start_time : datetime.datetime
    end_time : datetime.datetime
    nodes : list
        list of nodes to download
    node_names : list
        keys for dictionary (will be variables names in mat file)
        should be same length as nodes
    fn : str
        filename to save mat file to
    h_type : {'bb', 'lf'}
        type of hydropone (broadband, lowfrequency)
    verbose : bool
    
    '''
    

    if verbose:
        print('downloading data...')
    
    data = []
    if h_type == 'bb':
        for node in tqdm(nodes, disable=~verbose):
            data.append(ooipy.get_acoustic_data(start_time, end_time, node))
        
        if verbose:
            print('decimating broadband data...')
                          
        for x in data:
            try:
                x.decimate(8).decimate(8)
            except AttributeError:
                continue
    
    elif h_type == 'lf':
        for node in tqdm(nodes, disable=~verbose):
            data.append(ooipy.get_acoustic_data_LF(start_time, end_time, node))
    else:
        raise Exception(f"invalid h_type {h_type}, must be 'bb' or 'lf'")
        
    # create matlab readable datetimes
    attrs = []
    for station in data:
        try:
            attr = dict(station.stats)
        except AttributeError:
            attrs.append(None)
            continue
        
        attr['starttime'] = attr['starttime'].strftime('%Y-%m-%dT%H:%M:%S.%f')
        attr['endtime'] = attr['endtime'].strftime('%Y-%m-%dT%H:%M:%S.%f')

        attrs.append(attr)
                          
    # Save to mat file
    ## construct dictionary
    names = []
    mat_data = []
    for k, name in enumerate(node_names):
        names.append(name)
        names.append(name+'_stats')
        try:
            mat_data.append(data[k].data)
        except AttributeError:
            mat_data.append(None)
        mat_data.append(attrs[k])

    mat_dict = dict(zip(names, mat_data))
    mat_dict_nonone = {k: v for k, v in mat_dict.items() if v is not None}
    
    io.savemat(fn, mat_dict_nonone)
    
    return

In [69]:
start_time = datetime(2022,10,28)
end_time = datetime(2022,10,28,0,0,1)
nodes = ['LJ01C', 'PC01A', 'PC03A']

convert2mat(start_time, end_time, nodes, nodes, 'test.mat', h_type='bb')

downloading data...
decimating broadband data...


### October 2022

In [2]:
start_time = datetime(2022,10,28,2)
end_time = datetime(2022,10,28,4)

In [3]:
nodes_LF = ['Eastern_Caldera', 'Central_Caldera', 'Southern_Hydrate']
nodes_BB = ['LJ01C', 'PC01A', 'PC03A']

In [17]:
bb_data = []
for node in tqdm(nodes_BB):
    bb_data.append(ooipy.get_acoustic_data(start_time, end_time, node, verbose=True))

  0%|                                                                                                                                              | 0/3 [00:00<?, ?it/s]

Fetching URLs...
Sorting valid URLs for Time Window...
Removed large data gap at end of requested time


 33%|████████████████████████████████████████████▋                                                                                         | 1/3 [00:13<00:26, 13.03s/it]

Fetching URLs...
Sorting valid URLs for Time Window...
Removed large data gap at end of requested time


 67%|█████████████████████████████████████████████████████████████████████████████████████████▎                                            | 2/3 [00:26<00:13, 13.19s/it]

Fetching URLs...
Sorting valid URLs for Time Window...
Removed large data gap at end of requested time


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:38<00:00, 12.76s/it]


In [18]:
bb_data_dec = []
for station in tqdm(bb_data):
    bb_data_dec.append(station.decimate(8).decimate(8))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.14s/it]


In [51]:
# create matlab readable datetimes
attrs_bb = []
for station in bb_data_dec:
    attr_bb = dict(station.stats)
    
    attr_bb['starttime'] = attr_bb['starttime'].strftime('%Y-%m-%dT%H:%M:%S.%f')
    attr_bb['endtime'] = attr_bb['endtime'].strftime('%Y-%m-%dT%H:%M:%S.%f')
    
    attrs_bb.append(attr_bb)

In [52]:
names = ['oregon_offshore', 'oregon_offshore_stats', 'slope_base_200m', 'slope_base_200m_stats', 'axial_base_200m', 'axial_base_200m_stats']
data = [bb_data_dec[0].data, attrs_bb[0], bb_data_dec[1].data, attrs_bb[1], bb_data_dec[2].data, attrs_bb[2]]

mat_dict = dict(zip(names, data))

io.savemat('oct_2022_broadband.mat', mat_dict)

In [41]:
lf_data = []
for node in tqdm(nodes_LF):
    lf_data.append(ooipy.get_acoustic_data_LF(start_time, end_time, node, verbose=True))

  0%|                                                                                                                                              | 0/3 [00:00<?, ?it/s]

 33%|████████████████████████████████████████████▋                                                                                         | 1/3 [00:12<00:24, 12.31s/it]

 67%|█████████████████████████████████████████████████████████████████████████████████████████▎                                            | 2/3 [00:24<00:12, 12.05s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:36<00:00, 12.18s/it]


In [53]:
# create matlab readable datetimes
attrs_lf = []
for station in lf_data:
    attr_lf = dict(station.stats)
    
    attr_lf['starttime'] = attr_lf['starttime'].strftime('%Y-%m-%dT%H:%M:%S.%f')
    attr_lf['endtime'] = attr_lf['endtime'].strftime('%Y-%m-%dT%H:%M:%S.%f')
    
    attrs_lf.append(attr_lf)

In [54]:
names = ['eastern_caldera', 'eastern_caldera_stats', 'central_caldera', 'central_caldera_stats', 'southern_hydrate', 'southern_hydrate_stats']
data = [lf_data[0].data, attrs_lf[0], lf_data[1].data, attrs_lf[1], lf_data[2].data, attrs_lf[2]]

mat_dict = dict(zip(names, data))

io.savemat('oct_2022_lowfrequency.mat', mat_dict)

### December 2022

In [104]:
for day in range(1,5):

    start_time = datetime(2022,12,day,11)
    end_time = datetime(2022,12,day,13)

    nodes = ['LJ01C', 'PC01A', 'PC03A']
    names = ['oregon_offshore', 'slope_base_200m', 'axial_base_200m']
    fn = f'2022_12_{day}_1100_broadband.mat'

    convert2mat(start_time, end_time, nodes, names, fn, h_type='bb', verbose=True)
    
    start_time = datetime(2022,12,day,23)
    end_time = datetime(2022,12,day+1,1)
    fn = f'2022_12_{day}_2300_broadband.mat'

    convert2mat(start_time, end_time, nodes, names, fn, h_type='bb', verbose=True)

downloading data...
decimating broadband data...
downloading data...
decimating broadband data...
downloading data...
decimating broadband data...
downloading data...
decimating broadband data...
downloading data...
decimating broadband data...
downloading data...
decimating broadband data...
downloading data...
decimating broadband data...
downloading data...
decimating broadband data...


In [108]:
for day in range(1,5):

    start_time = datetime(2022,12,day,11)
    end_time = datetime(2022,12,day,13)

    nodes = ['Eastern_Caldera', 'Central_Caldera', 'Southern_Hydrate']
    names = ['easter_caldera', 'central_caldera', 'southern_hydrate']
    fn = f'2022_12_{day}_1100_lowfrequency.mat'

    convert2mat(start_time, end_time, nodes, names, fn, h_type='lf', verbose=True)
    
    start_time = datetime(2022,12,day,23)
    end_time = datetime(2022,12,day+1,1)
    fn = f'2022_12_{day}_2300_lowfrequency.mat'
    
    convert2mat(start_time, end_time, nodes, names, fn, h_type='lf', verbose=True)

downloading data...
downloading data...
downloading data...
downloading data...
downloading data...
downloading data...
downloading data...
downloading data...
